# Random search

In [46]:
import sympy as sp
import numpy as np

LIMITS_RANGE = -3, 3

COMPONENTS = 3
DATA_SIZE = 10

In [47]:
syms = sp.symbols(f'x1:{COMPONENTS+1}')

In [48]:
import operator
import functools

x1 = syms[0]
x2 = syms[1]
x3 = syms[2]

eq = sum(syms)
# eq = eq.subs(x1, 1/x1**2).subs(x2, x2**2).subs(x3, x3**3)
# eq = functools.reduce(operator.mul, syms, 1)

eq

x1 + x2 + x3

In [49]:
f = sp.lambdify(syms, eq, modules='numpy')

In [116]:
import typing
import math
import dataclasses

T = typing.TypeVar("T", int, float)
Vector = typing.Tuple[T, T]
VectorComponent = typing.List[T] | typing.Tuple[T, ...]
VectorArray = typing.List[typing.Tuple[T, T]]

def check_axis_intersect(that: Vector, z: T):
    x, y = that
    return y > z > x


def is_collision(that: VectorArray, other: VectorComponent) -> bool:
    if len(that) != len(other):
        raise ValueError("Invalid arguments, different length of values")
        
    return all([check_axis_intersect(t, z) for t, z in zip(that, other)])


@dataclasses.dataclass
class VectorStructure:
    data: VectorComponent
    value: T
    
    def __gt__(self, other):        
        return self.value > other.value
    
    def __str__(self):
        return f"{self.data}, {self.value}"
    
    def __iter__(self):
        return iter([*self.data, self.value])


def create_node_container_child(_array: VectorArray) -> VectorArray:
    all_data = []
    
    for arr in _array:
        x, y = arr
        d = math.floor(y - x) / 2
        
        all_data.append((x, x + d))
        all_data.append((x + d, y))
    
    return all_data


class NodeContainer:
    def __init__(self, data: VectorArray):
        self.__data = data
        self.__children: typing.List[NodeContainer] = [
            
        ]


class NodeData:
    def __init__(self):
        pass


class TreeNode:
    def __init__(self):
        pass
    
create_node_container_child([(-1, 1), (1, 1)])

[(-1, 0.0), (0.0, 1), (1, 1.0), (1.0, 1)]

In [57]:
import pandas as pd

def tree_search_algorithm(n_it=500):
    all_data = []
    
    for it in range(n_it):
        data = [np.random.uniform(-3, 3) for _ in range(COMPONENTS)]
        value = f(*data)
                
        all_data.append(
            VectorStructure(data, value)
        )
    
    return [list(vec) for vec in all_data], min(all_data)

all_data, min_found = tree_search_algorithm()

In [53]:
pd.DataFrame(all_data)

,0,1,2,3
0,0.993532,1.380039,-1.895430,0.478141
1,2.739897,1.802119,-0.447818,4.094198
2,1.705325,0.817082,-2.077405,0.445002
3,-0.924017,1.085211,0.163849,0.325043
4,-1.656772,1.896200,2.722122,2.961550
...,...,...,...,...
495,-1.952002,-1.728350,-0.883043,-4.563395
496,2.765692,0.342431,-2.320043,0.788080
497,0.498080,0.375626,-1.588905,-0.715200
498,2.689740,-0.519988,0.660932,2.830684


In [54]:
min_found

VectorStructure(data=[-2.9747211752535563, -2.270857012158328, -2.8904657979068005], value=-8.136043985318684)